# Compute to Data for Generative Art
This notebook is a continuation of the C2D notebook but this time specifically for the Cryptopunks generative art project.

Note: the code is currently not going to run, the goal of this notebook is to provide a template that can be used for the generative art GAN and any subsequent projects

### Notes from the original C2D Notebook:
- **3. Alice publishes a dataset** is not needed here (this is relevant for teams who want to publish datasets, not teams who want to use existing datasets to train ML models)
- **4. Alice publishes an algorithm** is relevant (we're going to publish the ML model as an algorithm)
- **5. Bob acquires datatokens for data and algorithm** is only needed to get the datatokens for the data (we already own the tokens for the algorithm)
- **6. Bob starts a compute job** is relevant (we need to specify where we want to train the algorithm on. Initially, the simplest option is connecting to an AWS compute instance, later on to decentralized computer providers such as the Raven Protocol)
- **7. Bob monitors logs / algorithm output** is relevant

#### Additional steps not covered in C2D Notebook:
- testing the trained model
- running inference
- retraining the model (after change of architecture/hyperparameter tuning)
- publishing the trained model on the marketplace
- using the model from the marketplace to get a new Cryptopunk

## Publishing the Generative Art algorithm

In [1]:
# Important Parameters
wallet: str # our wallet address (for having ownership of the algorithm & buying dataset from Ocean Market)
entrypoint: str # The Docker entrypoint. $ALGO is a macro that gets replaced inside the compute job, depending where your algorithm code is downloaded.
image: str # the Docker image name
tag: str # the Docker image tag
url: str = "https://raw.githubusercontent.com/AlgoveraAI/generative-art/main/cp_gan.py"
name: str = "cp_gan"
author: str = "Algovera"
    


In [ ]:
# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('ALG1', 'ALG1', wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(wallet.address, to_wei(100), wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": entrypoint,
              "image": image,
              "tag": tag
            }
        },
        "files": [
      {
        "url": url,
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": name, "author": author, "license": "CC0",
    "dateCreated": "2020-01-28T10:55:11Z"
    }
}
ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

## Getting Datatokens for the Cryptopunks dataset
The C2D notebook had the private keys of both Alice and Bob. This time, we just have our own private key and we want to buy datatokens for the [Cryptopunks dataset](https://market.oceanprotocol.com/asset/did:op:C9D0568838fa670baEe7195Ea443b32EfCAc2281).

For this, we use the [Ocean Market flow](https://github.com/oceanprotocol/ocean.py/blob/main/READMEs/marketplace-flow.md).

Note: Take a look at the Cryptopunks datatoken [here](https://rinkeby.etherscan.io/token/0xC9D0568838fa670baEe7195Ea443b32EfCAc2281)

In [ ]:
# Important Parameters

# from the previous notebooks, your environment should store some test private keys
private_key: str = os.getenv('TEST_KEY') # note: do NOT store your real private keys anywhere on your machine
token_address: str = "0xC9D0568838fa670baEe7195Ea443b32EfCAc2281" # address of the Cryptopunks datatoken

In [ ]:
wallet = Wallet(ocean.web3, private_key, config.block_confirmations, config.transaction_timeout)
print(f"wallet.address = '{wallet.address}'")

#Verify that we have ganache ETH
assert ocean.web3.eth.get_balance(wallet.address) > 0, "need ganache ETH"

#Verify that we have ganache OCEAN
assert OCEAN_token.balanceOf(wallet.address) > 0, "need ganache OCEAN"

#We buy 1.0 datatokens - the amount needed to consume the dataset.
data_token = ocean.get_data_token(token_address)
ocean.pool.buy_data_tokens(
    pool_address,
    amount=to_wei(1), # buy 1.0 datatoken
    max_OCEAN_amount=to_wei(10), # pay up to 10.0 OCEAN
    from_wallet=wallet
)

# Check we have the datatoken
from ocean_lib.web3_internal.currency import pretty_ether_and_wei
print(f"I have {pretty_ether_and_wei(data_token.balanceOf(wallet.address), data_token.symbol())}.")

assert data_token.balanceOf(wallet.address) >= to_wei(1), "I didn't get 1.0 datatokens"

## Running a compute job on the dataset
Now that we have the datatoken for the Cryptopunks dataset and the datatokens for our algorithm, we can run a compute job.

In [ ]:
DATA_did = DATA_ddo.did  # for convenience
ALG_did = ALG_ddo.did
DATA_DDO = ocean.assets.resolve(DATA_did)  # make sure we operate on the updated and indexed metadata_cache_uri versions
ALG_DDO = ocean.assets.resolve(ALG_did)

compute_service = DATA_DDO.get_service('compute')
algo_service = ALG_DDO.get_service('access')

from ocean_lib.web3_internal.constants import ZERO_ADDRESS
from ocean_lib.models.compute_input import ComputeInput

# order & pay for dataset
dataset_order_requirements = ocean.assets.order(
    DATA_did, wallet.address, service_type=compute_service.type
)
DATA_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        dataset_order_requirements.amount,
        dataset_order_requirements.data_token_address,
        DATA_did,
        compute_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        dataset_order_requirements.computeAddress,
    )

# order & pay for algo (we're the owner so this might be a bit different)
algo_order_requirements = ocean.assets.order(
    ALG_did, wallet.address, service_type=algo_service.type
)
ALG_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        algo_order_requirements.amount,
        algo_order_requirements.data_token_address,
        ALG_did,
        algo_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        algo_order_requirements.computeAddress,
)

Now we need to specify the compute input.

In [ ]:
compute_inputs = [ComputeInput(DATA_did, DATA_order_tx_id, compute_service.index)]
job_id = ocean.compute.start(
    compute_inputs,
    bob_wallet,
    algorithm_did=ALG_did,
    algorithm_tx_id=ALG_order_tx_id,
    algorithm_data_token=ALG_datatoken.address
)
print(f"Started compute job with id: {job_id}")

## Logging the outputs

In [ ]:
ocean.compute.status(DATA_did, job_id, bob_wallet) # rerun multiple times until output {'ok': True, 'status': 70, 'statusText': 'Job finished'}

In [ ]:
result = ocean.compute.result_file(DATA_did, job_id, 1, bob_wallet)  # 0 index, means we retrieve the results from the first dataset index

## Model test, inference, improvements
WIP